# Tutorial: LabDataForester

Here is a demo tutorial for a large nested dataset. 

You need:
1. A running database management system (DBMS), most likely on Neo4j Desktop
2. This iPython notebook, running at the base of the `labdataforester` repository

There is a little bit of work to do before step one...
- Download and install Neo4j Desktop Community Edition here: https://neo4j.com/deployment-center/#:~:text=Neo4j%20Desktop,to%20get%20started.
- Follow some more specific instructions here: https://wikis.mit.edu/confluence/display/KIPIT/Connecting+to+the+Knowledge+Graph

Once these things are set, this notebook will guide you through the example of loading the repository's filesystem into a knowledge graph for navigation.

## Surveying a Lab's Data Forest

In this notebook, you can see how a large messy set of imaging data can be surveyed, modeled, and pushed into a running database. 

## Large datasets

Python's glob library struggles with large imaging datasets since there are many files to scan and many nested datasets. When this in the case, it helps to break down glob's scope. Here is a demonstration of how to deal with a large dataset.

First, process the directories in parallel, creating checkpoints in `.pkl` format. 

Then, open each checkpoint and push to the database.

In [4]:
import os
import glob
import concurrent.futures
from pathlib import Path
import labdataforester as ldf
from neo4j import GraphDatabase
import .labdataranger as ldr


def process_directory(
    base_directory_path,
    skips=['System Volume Information','$RECYCLE.BIN'],
    checkpoint_fstr='tree.pkl',
    log_fstr='tree.out',
    verbose=False):
    
    checkpoint_file = os.path.join(base_directory_path, checkpoint_fstr)
    log_file = os.path.join(base_directory_path, log_fstr)
    
    if verbose:
        print(base_directory_path)
        print(checkpoint_file)
        print(log_file)
        print()

    ft = ldf.survey.FileTree(
        base_directory_path,
        skips, 
        log_file=log_file, 
        checkpoint_file=checkpoint_file        
    )
    ft.collect_file_tree()
    ft.save_state(checkpoint_file)

    return base_directory_path


def process_parallel(base_path, 
                     max_workers=56, 
                     verbose=False):

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    
        futures = {
            executor.submit(process_directory, base_dir): {
                'base': base_dir, 'name': dir_name
            } for dir_name, base_dir 
            in ldf.survey.get_base_dirs(base_path).items()
        }
        
    
        for future in concurrent.futures.as_completed(futures):
        
            _dir = futures[future]
            
            try:
                result = future.result()
                if verbose:
                    print(f"Processing completed for: {result}")
                    print(f"                          {_dir['name']}")
                    
            except Exception as exc:
                print(f"Error processing {_dir['base']}: {exc}")


def push_forest_to_db(base_path, 
                      checkpoint_fstr='tree.pkl'):
    
    for dir_name, base_dir in ldf.survey.get_base_dirs(base_path).items():
        print(dir_name)
        checkpoint = Path(base_dir).joinpath(checkpoint_fstr)
        ft = ldf.survey.FileTree(base_dir, checkpoint_file=checkpoint)    
        class_map, class_dict = ldf.model.build_classes(ft.graph)
        try:
            node_map = ldf.store.push_to_neo4j(ft.graph, class_map)
        except: 
            print(f"ERROR: {dir_name} filetree could not be loaded")



def create_and_connect_folders(parent_folder_path, 
                               database=None,
                               config_file='db_config.json'):

    db_config = ldr.query.get_db_config(config_file=config_file)
    uri = f"{db_config['uri']}:{db_config['port']}"
    username = db_config['username']
    password = db_config['password']
    
    driver = GraphDatabase.driver(uri, auth=(username, password))
    parent_folder_name = os.path.basename(parent_folder_path.rstrip('/'))    

    query = """
        MERGE (parent:Folder {filepath: $parentFolderPath, name: $parentFolderName})
        WITH parent
        MATCH (subfolder:Folder)
        WHERE subfolder.filepath STARTS WITH $parentFolderPath + '/'
          AND NOT (subfolder)<-[:CONTAINS_FOLDER]-(:Folder)
          AND subfolder.filepath <> $parentFolderPath
        MERGE (parent)-[:CONTAINS_FOLDER]->(subfolder)
        RETURN parent, subfolder
    """
    
    with driver.session(database=database) as session:
        
        result = session.run(query, 
                             parentFolderPath=parent_folder_path,
                             parentFolderName=parent_folder_name)
        
        print("Related Subfolders to Base Folder")
        for record in result:
            print()
            print(f"  Parent Folder: {record['parent']['filepath']}")
            print(f"      Subfolder: {record['subfolder']['filepath']}")
            print()
            
    driver.close()


In [2]:
%%time

database='instruments'
base_path = '/mnt/data/archive/LaraM_Full_Dataset'
ldf.store.configure_db(database=database)

process_parallel(base_path)
push_forest_to_db(base_path)
create_and_connect_folders(base_path, database=database)

State saved to /mnt/data/archive/LaraM_Full_Dataset/JnJ_MIT_uCTshare/tree.pkl.
State saved to /mnt/data/archive/LaraM_Full_Dataset/$RECYCLE.BIN/tree.pkl.
State saved to /mnt/data/archive/LaraM_Full_Dataset/System Volume Information/tree.pkl.
State saved to /mnt/data/archive/LaraM_Full_Dataset/1_post_Rec/tree.pkl.
State saved to /mnt/data/archive/LaraM_Full_Dataset/1.10.22_SW/tree.pkl.
Error reading file /mnt/data/archive/LaraM_Full_Dataset/4.06.22_SW_U26/327/._327.log: 'utf-8' codec can't decode byte 0xb0 in position 37: invalid start byte
State saved to /mnt/data/archive/LaraM_Full_Dataset/Sam_Normal mice/tree.pkl.
State saved to /mnt/data/archive/LaraM_Full_Dataset/11.09.21_SW_U19/tree.pkl.
State saved to /mnt/data/archive/LaraM_Full_Dataset/_Jnj/tree.pkl.
State saved to /mnt/data/archive/LaraM_Full_Dataset/4.06.22_SW_U26/tree.pkl.
State saved to /mnt/data/archive/LaraM_Full_Dataset/4.20.22_SW_U26(7)/tree.pkl.
State saved to /mnt/data/archive/LaraM_Full_Dataset/12.06.21_SW_U19_U20/tr

Edges: 100%|███████████████████████████████████████████████████| 67498/67498 [00:32<00:00, 2076.97it/s]


aexit called
Graph loading complete!
U52
State loaded from /mnt/data/archive/LaraM_Full_Dataset/U52/tree.pkl.
No graph file found at /mnt/data/archive/LaraM_Full_Dataset/U52/tree.graphml.
File path index built.
File tree graph built.
aenter called


Edges: 100%|███████████████████████████████████████████████████| 68004/68004 [00:30<00:00, 2233.62it/s]


aexit called
Graph loading complete!
03.17.22_SW
State loaded from /mnt/data/archive/LaraM_Full_Dataset/03.17.22_SW/tree.pkl.
No graph file found at /mnt/data/archive/LaraM_Full_Dataset/03.17.22_SW/tree.graphml.
File path index built.
File tree graph built.
aenter called


...

Edges: 100%|███████████████████████████████████████████████████| 21487/21487 [00:10<00:00, 2024.08it/s]


aexit called
Graph loading complete!
09.03.21_sw_U18
State loaded from /mnt/data/archive/LaraM_Full_Dataset/09.03.21_sw_U18/tree.pkl.
No graph file found at /mnt/data/archive/LaraM_Full_Dataset/09.03.21_sw_U18/tree.graphml.
File path index built.
File tree graph built.
aenter called


Edges: 100%|███████████████████████████████████████████████████| 12175/12175 [00:05<00:00, 2090.27it/s]


aexit called
Graph loading complete!


NameError: name 'driver' is not defined

### Push Trees into Forest

Next step is to push everything to the database.

Once scans are complete for the base directory, we can now link the subfolders to the base directory for a complete filetree.

Note: This can be applied iteratively to later collect entire directories to link them together.